In [ ]:
pip install numpy pandas matplotlib opencv-python torch torchvision facenet-pytorch

In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((160, 160)),  # resizing
    transforms.ToTensor(),  # transform the data to tensors
    transforms.Normalize([0.5], [0.5])  # normalization
])

lfw_dataset = datasets.LFWPeople(root="./data", split="train", transform=transform, download=True)
print(f"Dataset size: {len(lfw_dataset)} images")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# some random data to show
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i in range(5):
    image, label = lfw_dataset[i]
    image = image.permute(1, 2, 0).numpy()  
    axes[i].imshow((image * 0.5) + 0.5)  
    axes[i].set_title(f"Label: {label}")
    axes[i].axis("off")

plt.show()

In [ ]:
from torch.utils.data import DataLoader, random_split

train_size = int(0.8 * len(lfw_dataset))
test_size = len(lfw_dataset) - train_size

train_dataset, test_dataset = random_split(lfw_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# loading the model (pretrained)
model = models.resnet18(pretrained=True)

# chaning the last layer for lfw
num_features = model.fc.in_features
num_classes = len(set(lfw_dataset.targets)) 
model.fc = nn.Linear(num_features, num_classes)

model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")